# Lot Size Calculator

In [1]:
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy
import ta
#import talib
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5

# establish MetaTrader 5 connection to a specified trading account
if not mt5.initialize(login=50757251, server="ICMarketsSC-Demo",password="D8sBpykf"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [2]:
import MetaTrader5 as mt5
# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)
 
# establish MetaTrader 5 connection to a specified trading account
if not mt5.initialize(login=50757251, server="ICMarketsSC-Demo",password="D8sBpykf"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
# display data on connection status, server name and trading account
print(mt5.terminal_info())
# display data on MetaTrader 5 version
print(mt5.version())
 
# shut down connection to the MetaTrader 5 terminal
#mt5.shutdown()

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.37
TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=False, trade_allowed=True, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=True, build=3270, maxbars=100000, codepage=0, ping_last=182283, community_balance=0.0, retransmission=0.0, company='Raw Trading Ltd', name='MetaTrader 5 IC Markets (SC)', language='English', path='C:\\Users\\RaimiAzeez\\AppData\\Roaming\\MetaTrader 5 IC Markets (SC)', data_path='C:\\Users\\RaimiAzeez\\AppData\\Roaming\\MetaTrader 5 IC Markets (SC)', commondata_path='C:\\Users\\RaimiAzeez\\AppData\\Roaming\\MetaQuotes\\Terminal\\Common')
(500, 3270, '21 Apr 2022')


# Lot Size Calc

In [2]:
# get account currency
account_currency=mt5.account_info().currency
print("Account currency:",account_currency)

# get account balance 
account_info=mt5.account_info()
if account_info!=None:
    account_balance = account_info.balance
    print("Account Balance = $", account_balance)

# define risk amount
# risk in value
#risk = 1000 
#risk in percentage
risk_perc = 1 #%
risk = (account_balance * risk_perc)/100
print("Risk Amount = $", risk)

#get currency price
symbol = "EURUSD"
price =mt5.symbol_info(symbol).ask
print(symbol, "price = $", price )

#Define stop loss
stop_loss_pip = 200 #in pips
stop_loss = 0.72362 # In Value

#Calculate Currency per pair
curr_per_pair = risk/stop_loss_pip
print("Currency per pip = ", account_currency, curr_per_pair,"/pip")

# Calculate number of units
point=mt5.symbol_info(symbol).point
units = curr_per_pair * 10000
lot_size = units/100000
print("Units = ", units)
print("Lot Size1 = ", lot_size)

#Shortcut
points_short = stop_loss_pip * 10
lot = risk / points_short
print("Lot size = ", lot)


Account currency: USD
Account Balance = $ 92832.99
Risk Amount = $ 928.3299000000001
EURUSD price = $ 1.05371
Currency per pip =  USD 4.641649500000001 /pip
Units =  46416.49500000001
Lot Size1 =  0.4641649500000001
Lot size =  0.46416495


# Automated

In [15]:

"""
# get account currency
account_currency= str(input("Input account currency: "))
#print("Account currency:",account_currency)

# get account balance 
account_balance = float(input("Input Account Balance: "))
#print("Account Balance = $", account_balance)

#risk in percentage
risk_perc = float(input("Input risk amout in percentage: ")) #%
risk = (account_balance * risk_perc)/100
#print("Risk Amount = $", risk)

#Define stop loss
stop_loss_pip = float(input("Enter stop loss (in pips): ")) #in pips

#Calculate Lot size
points_short = stop_loss_pip * 10
lot = round(risk / points_short, 2)
print("Lot size = ", lot)

"""

Input account currency: USD
Input Account Balance: 3000
Input risk amout in percentage: 2
Enter stop loss (in pips): 20
Lot size =  0.3


In [16]:
def cal_lot_size(balance, risk, stop_loss):
    global lot, account_currency, account_balance, risk_perc, stop_loss_pip
    # connect to MetaTrader 5
    if not mt5.initialize():
        print("initialize() failed")
        mt5.shutdown()

    # get currency pair
    #pair = str(pair) #str(input("Instrument: "))
    #print("Account currency:",account_currency) 
    
    # get account currency
    account_currency= "USD" #str(input("Input account currency: "))
    #print("Account currency:",account_currency)

    # get account balance 
    account_balance = float(balance) #int(input("Input Account Balance: "))
    #print("Account Balance = $", account_balance)

    #risk in percentage
    risk_perc = float(risk) #int(input("Input risk amout in percentage: ")) #%
    risk = (account_balance * risk_perc)/100
    #print("Risk Amount = $", risk)

    #Define stop loss
    stop_loss_pip = float(stop_loss)   # int(input("Enter stop loss (in pips): ")) #in pips

    #Calculate Lot size
    #point = mt5.symbol_info(pair).point
    #points_short = stop_loss_pip * point
    lot = round(risk / points_short, 2)
    return lot
    #print("Lot size = ", lot)


In [18]:
cac = cal_lot_size(1000, 2, 30)
cac

0.1

# Flask Deployment

In [19]:
#importing the required libraries
from flask import Flask, render_template, request, redirect, url_for
from joblib import load
from flask_ngrok import run_with_ngrok
#from get_tweets import get_related_tweets
from logging import FileHandler,WARNING

In [20]:
import os
import threading

from flask import Flask
from pyngrok import ngrok

# Open a HTTP tunnel on the default port 80
public_url = ngrok.connect(port = '4040')

os.environ["FLASK_ENV"] = "development"

app = Flask(__name__)
port = 5000

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL


# Define Flask routes
# render default webpage
@app.route('/')
def home():
    return render_template("lot size home.html")

# when the post method detect, then redirect to success function
@app.route('/', methods=['POST', 'GET'])
def get_data():
    global acnt, bal, risk, stop
    if request.method == 'POST':
        #acnt = request.form['account_currency']
        bal = int(request.form['account_balance'])
        risk = int(request.form['risk_amount'])
        stop = int(request.form['stop_loss'])
        return redirect(url_for('success', name=stop))
    else:
      return render_template('lot size home.html')

# get the data for the requested query
@app.route('/success/<name>')
def success(name):
    lot = cal_lot_size(bal, risk, stop)
    
    return render_template("lot size home2.html", variable = lot)

 * ngrok tunnel "http://0500-41-184-210-11.ngrok.io" -> "http://127.0.0.1:5000"


t=2022-06-23T15:07:32+0100 lvl=warn msg="failed to open private leg" id=62e5ce3272f0 privaddr=localhost:5000 err="dial tcp [::1]:5000: connectex: No connection could be made because the target machine actively refused it."
t=2022-06-23T15:07:34+0100 lvl=warn msg="failed to open private leg" id=31783b6a2d38 privaddr=localhost:5000 err="dial tcp [::1]:5000: connectex: No connection could be made because the target machine actively refused it."


In [21]:
# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: development
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Jun/2022 15:07:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2022 15:08:03] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Jun/2022 15:08:19] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [23/Jun/2022 15:08:20] "GET /success/150 HTTP/1.1" 200 -


In [22]:
ngrok.kill()